# Markov chain Monte Carlo: Amortized Approximate Likelihood Ratios

In [1]:
import hypothesis
import torch
import numpy as np
import matplotlib.pyplot as plt

## Forward model

In [2]:
from hypothesis.simulation import Simulator

class NormalSimulator(Simulator):
    
    def __init__(self):
        super(NormalSimulator, self).__init__()
        
    def forward(self, inputs):
        return torch.randn(inputs.size(0), 1) + inputs

simulator = NormalSimulator()

## Prior

In [3]:
from torch.distributions.uniform import Uniform

prior = Uniform(-30, 30)

## Ratio estimator architecture

In [4]:
from hypothesis.nn import BaseConditionalRatioEstimator

class RatioEstimator(BaseConditionalRatioEstimator):
    
    def __init__(self):
        super(RatioEstimator, self).__init__()
        self.network = torch.nn.Sequential(
            torch.nn.Linear(2, 64),
            torch.nn.ELU(),
            torch.nn.Linear(64, 64),
            torch.nn.ELU(),
            torch.nn.Linear(64, 64),
            torch.nn.ELU(),
            torch.nn.Linear(64, 1))
        
    def forward(self, inputs, outputs):
        log_ratio = self.log_ratio(inputs, outputs)
        
        return log_ratio.sigmoid(), log_ratio
        
    def log_ratio(self, inputs, outputs):
        inputs = inputs.view(-1, 1)
        outputs = outputs.view(-1, 1)
        x = torch.cat([inputs, outputs], dim=1)
    
        return self.network(x)

## Training the ratio estimator

In [ ]:
from hypothesis.util.data import SimulatorDataset
from hypothesis.nn.conditional_ratio_estimator import ConditionalRatioEstimatorCriterion

ratio_estimator = RatioEstimator()
dataset = SimulatorDataset(simulator, prior)
batch_size = 128000
criterion = ConditionalRatioEstimatorCriterion(ratio_estimator, batch_size)
optimizer = torch.optim.Adam(ratio_estimator.parameters())
epochs = 25

losses = []
for epoch in range(epochs):
    data_loader = torch.utils.data.DataLoader(dataset, num_workers=2, batch_size=batch_size, drop_last=True)
    num_batches = len(data_loader)
    data_loader = iter(data_loader)
    for batch_index in range(num_batches):
        optimizer.zero_grad()
        inputs, outputs = next(data_loader)
        loss = criterion(inputs, outputs)
        loss.backward()
        optimizer.step()
    losses.append(loss.cpu().item())
    
losses = np.array(losses)
plt.plot(np.arange(epochs), np.log(losses), lw=2, color="black")
plt.show()

In some cases the criterion based on the logits might be more numerically preferable.

In [5]:
from hypothesis.nn.conditional_ratio_estimator import ConditionalRatioEstimatorLogitsCriterion

ratio_estimator = RatioEstimator()
dataset = SimulatorDataset(simulator, prior)
batch_size = 128000
criterion = ConditionalRatioEstimatorLogitsCriterion(ratio_estimator, batch_size)
optimizer = torch.optim.Adam(ratio_estimator.parameters())
epochs = 25

losses = []
for epoch in range(epochs):
    data_loader = torch.utils.data.DataLoader(dataset, num_workers=2, batch_size=batch_size, drop_last=True)
    num_batches = len(data_loader)
    data_loader = iter(data_loader)
    for batch_index in range(num_batches):
        optimizer.zero_grad()
        inputs, outputs = next(data_loader)
        loss = criterion(inputs, outputs)
        loss.backward()
        optimizer.step()
    losses.append(loss.cpu().item())
    
losses = np.array(losses)
plt.plot(np.arange(epochs), np.log(losses), lw=2, color="black")
plt.show()

NameError: name 'SimulatorDataset' is not defined

## Validating the ratio estimator

## Posterior inference using MCMC